In [2]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn

df = pd.read_csv('/workspace/nabang1010/LBA_VAIPE/GNN/Multi_Label_Image_Recognition_with_Graph_Convolutional_Networks/data/public_train/id_mapping.csv')
df.sample(3)

,drugname,mapping
197,SETBLOOD 115mg+100mg+50mcg,104
159,NIFEDIPIN HASAN 20 RETARD 20mg,84
82,"CEFADROXIL 500MG 0,5g",43


In [9]:
id_list = []
id_name = []

for id in range(107):
    df_mini = df[df["mapping"] == id]
    
    name_cat = " "
    for name in df_mini["drugname"]:
        name_cat += name + " "
    id_list.append(id)
    id_name.append(name_cat)
df_save = pd.DataFrame({"id": id_list, "drugname": id_name})
# df_save.to_csv("/workspace/nabang1010/LBA_VAIPE/GNN/Multi_Label_Image_Recognition_with_Graph_Convolutional_Networks/data/public_train/id_mapping_cat.csv", index=False)

In [47]:
# addd 1 line to the end of the df_save
df_save = df_save.append({"id": 107, "drugname": "MOT LINH BAY"}, ignore_index=True)

In [48]:
df_save.sample(5)

,id,drugname,drugname_embedding
46,46,ENALAPRIL 5mg,"[[0.10151192545890808, -0.2622784674167633, 0...."
3,3,MEDIPLEX 800mg,"[[-0.3559682369232178, -0.16712109744548798, 0..."
89,89,MYPARA 500 500mg PARTAMOL TAB. 500mg PANACTOL...,"[[-0.034637682139873505, -0.1025574654340744, ..."
40,40,DICLOFENAC 50mg Loxoprofen (Mezafen) 60mg MEZ...,"[[-0.026793386787176132, -0.012113688513636589..."
64,64,"Dinh lang, bach qua (Hoat huyet duong nao) 15...","[[-0.106545090675354, -0.029003435745835304, 0..."


In [49]:
# TO_CUDA = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# get embedding from bert multilingual cased 
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased")
# input is df_save["drugname"]
# output is df_save["drugname_embedding"]
token = tokenizer.batch_encode_plus(df_save["drugname"].tolist(), return_tensors='pt', padding=True, truncation=True)
output = model(**token)
df_save["drugname_embedding"] = output[0].detach().numpy().tolist()



Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [66]:
bert_relationship = output["pooler_output"]

In [67]:
bert_relationship = bert_relationship.detach().numpy()


In [69]:
# save bert_relationship to pickle file
import pickle
with open('/workspace/nabang1010/LBA_VAIPE/GNN/Multi_Label_Image_Recognition_with_Graph_Convolutional_Networks/data/public_train/bert_relationship_768.pickle', 'wb') as f:
    pickle.dump(bert_relationship, f)
